<a href="https://colab.research.google.com/github/Ximenaloranca/Ecuaciones-Diferenciales-Parciales-/blob/main/Metodo_de_Galerkin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Definimos las variables
x = sp.symbols('x')
u = sp.Function('u')

ode = -u(x).diff(x, x) + u(x) - x # Escribimos la ecuacion

condiciones = {u(0): 0, u(1): 0} # Ponemos las condiciones de frontera

# Solucion exacta
sol_exacta = sp.dsolve(ode, u(x), ics=condiciones)
u_exacta_expr = sol_exacta.rhs

sp.pprint(u_exacta_expr) # Imprimimos la solucion exacta
# Yo no se porque se sigue vienddo fea

In [ ]:
a1, a2, a3 = sp.symbols('alpha_1 alpha_2 alpha_3') # Definimos los coeficiente

beta = sp.pi # Hacemos que beta valga pi, para que se puedan cumplir las condiciones

# Hacemos las funciones
func_beta_1 = sp.sin(1 * beta * x)
func_beta_2 = sp.sin(2 * beta * x)
func_beta_3 = sp.sin(3 * beta * x)

u_approx = a1*func_beta_1 + a2*func_beta_2 + a3*func_beta_3 # Construimos una funcion para la combinacion lineal

sp.pprint(u_approx)

R = -sp.diff(u_approx, x, 2) + u_approx - x # Definimos el error

eq1 = sp.integrate(R * func_beta_1, (x, 0, 1)) # Aplicamos ya el metodo y hacemos las ecuaciones
eq2 = sp.integrate(R * func_beta_2, (x, 0, 1))
eq3 = sp.integrate(R * func_beta_3, (x, 0, 1))

print(f"Eq 1: {eq1} = 0")
print(f"Eq 2: {eq2} = 0")
print(f"Eq 3: {eq3} = 0")

# Resolvemos el sistema lineal
sol_alphas = sp.solve([eq1, eq2, eq3], [a1, a2, a3])

sp.pprint(sol_alphas)

# Sustituimos los coeficientes
u_galerkin = u_approx.subs(sol_alphas)

print("Solucion aproximada")
sp.pprint(u_galerkin)

Hacemos las graficas para ver que pasa con las soluciones

In [ ]:
f_exacta = sp.lambdify(x, u_exacta_expr, modules="numpy")
f_galerkin = sp.lambdify(x, u_galerkin, modules="numpy")

x_vals = np.linspace(0, 1, 100)
y_exacta = f_exacta(x_vals)
y_galerkin = f_galerkin(x_vals)

plt.figure(figsize=(10, 6))
plt.plot(x_vals, y_exacta, 'k-', linewidth=3, alpha=0.5, label='Solución Exacta')
plt.plot(x_vals, y_galerkin, 'r--', linewidth=1.5, label=r'Aproximación ($\sin(i \beta x)$)')

plt.title(r"Comparación: Exacta vs Galerkin (usando $\beta=\pi$)")
plt.xlabel('x')
plt.ylabel('u(x)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:

error_max = np.max(np.abs(y_exacta - y_galerkin))
error_sq = (u_exacta_expr - u_galerkin)**2
error_l2 = sp.sqrt(sp.integrate(error_sq, (x, 0, 1))).evalf()

print(f"Error máximo puntual: {error_max:.6e}")
print(f"Error norma L2: {error_l2:.6e}")